In [1]:
import cv2
import numpy as np
import os
import torch
import torch.nn.functional as F
from torchvision.transforms import Compose
from tqdm import tqdm
from torchvision.io import read_image, ImageReadMode

#from depth_anything.dpt import DepthAnything
#from depth_anything.util.transform import Resize, NormalizeImage, PrepareForNet

import matplotlib.pyplot as plt

In [2]:
from zoedepth.utils.easydict import EasyDict as edict
#from zoedepth.data.data_mono import DepthDataLoader
from zoedepth.models.builder import build_model
from zoedepth.utils.arg_utils import parse_unknown
from zoedepth.utils.config import change_dataset, get_config, ALL_EVAL_DATASETS, ALL_INDOOR, ALL_OUTDOOR
from zoedepth.utils.misc import (RunningAverageDict, colors, compute_metrics,count_parameters)

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DEVICE

'cuda'

In [4]:
pretrained_resource="local::./checkpoints/depth_anything_metric_depth_outdoor.pt"
m_name = "zoedepth"

In [5]:
overwrite = {"pretrained_resource": pretrained_resource}
config = get_config(m_name, "eval", 'nyu', **overwrite)
print(config)

{'dataset': 'nyu', 'avoid_boundary': False, 'min_depth': 0.001, 'max_depth': 10, 'data_path': './data/nyu', 'gt_path': './data/nyu', 'filenames_file': './train_test_inputs/nyudepthv2_train_files_with_gt.txt', 'input_height': 480, 'input_width': 640, 'data_path_eval': './data/nyu', 'gt_path_eval': './data/nyu', 'filenames_file_eval': './train_test_inputs/nyudepthv2_test_files_with_gt.txt', 'min_depth_eval': 0.001, 'max_depth_eval': 10, 'min_depth_diff': -10, 'max_depth_diff': 10, 'do_random_rotate': True, 'degree': 1.0, 'do_kb_crop': False, 'garg_crop': False, 'eigen_crop': True, 'save_dir': './depth_anything_finetune', 'project': 'ZoeDepth', 'tags': '', 'notes': '', 'gpu': None, 'root': '.', 'uid': None, 'print_losses': False, 'distributed': True, 'workers': 16, 'clip_grad': 0.1, 'use_shared_dict': False, 'shared_dict': None, 'use_amp': False, 'aug': True, 'random_crop': False, 'random_translate': False, 'translate_prob': 0.2, 'max_translation': 100, 'validate_every': 0.25, 'log_images

In [6]:
model = build_model(config)


Params passed to Resize transform:
	width:  518
	height:  392
	resize_target:  True
	keep_aspect_ratio:  False
	ensure_multiple_of:  14
	resize_method:  minimal
Using pretrained resource local::./checkpoints/depth_anything_metric_depth_outdoor.pt
Loaded successfully


In [7]:
img_pose_file = "/home/jason/ROS/bags/analysis/pose.txt"
img_pose = list()

f = open(img_pose_file, "r")
content = f.readlines()

frame1 = content[75].split(',')
frame2 = content[76].split(',')

frame1_pose = (float(frame1[1]), float(frame1[2]), float(frame1[3]))
frame1_img = frame1[0]

frame2_pose = (float(frame2[1]), float(frame2[2]), float(frame2[3]))
frame2_img = frame2[0]

img1 = read_image("/home/jason/ROS/bags/images/wp_ground/"+frame1_img, mode=ImageReadMode.RGB)
img2 = read_image("/home/jason/ROS/bags/images/wp_ground/"+frame2_img, mode=ImageReadMode.RGB)

In [8]:
model.eval()

ZoeDepth(
  (core): DepthAnythingCore(
    (core): DPT_DINOv2(
      (pretrained): DinoVisionTransformer(
        (patch_embed): PatchEmbed(
          (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
          (norm): Identity()
        )
        (blocks): ModuleList(
          (0-23): 24 x NestedTensorBlock(
            (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (attn): MemEffAttention(
              (qkv): Linear(in_features=1024, out_features=3072, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=1024, out_features=1024, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): LayerScale()
            (drop_path1): Identity()
            (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
            (mlp): Mlp(
              (fc1): Linear(in_features=1024, out_features=4096, bias=True)
              (act): GELU(approximat

In [16]:
def infer(model, img, **kwargs):
    pred = model(img, **kwargs)
    return get_depth_from_prediction(pred)

def get_depth_from_prediction(pred):
    if isinstance(pred, torch.Tensor):
        pred = pred  # pass
    elif isinstance(pred, (list, tuple)):
        pred = pred[-1]
    elif isinstance(pred, dict):
        pred = pred['metric_depth'] if 'metric_depth' in pred else pred['out']
    else:
        raise NotImplementedError(f"Unknown output type {type(pred)}")
    return pred

In [17]:
focal = torch.Tensor([(879.2263626388094 + 879.3081232332211) / 2])

img_t = img1.unsqueeze(0)
print(img_t.shape, img1.dtype)
img_t = img_t.type(torch.float)

pred = infer(model.cuda(), img_t.cuda(), focal=focal.cuda() )

torch.Size([1, 3, 720, 1280]) torch.uint8


In [29]:
from zoedepth.utils.misc import colorize

depth = pred.squeeze(0,1).cpu()

d_map = colorize(depth.detach().numpy(), 0, 10)
plt.imshow(depth.detach().numpy())

ValueError: Input and output must have the same number of spatial dimensions, but got input with spatial dimensions of [518] and output size of (720, 1280). Please provide input tensor in (N, C, d1, d2, ...,dK) format and output size in (o1, o2, ...,oK) format.